In [ ]:
import math
#from langdetect import detect,detect_langs
import validators

import pandas as pd
import numpy as np
import os
import email
import email.policy
from bs4 import BeautifulSoup
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import RegexpTokenizer,word_tokenize
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix
from string import punctuation
from gensim.parsing.preprocessing import STOPWORDS
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import re
import heapq

import matplotlib.pyplot as plt

In [ ]:
os.listdir('./Database')

In [ ]:
arquivosNormais = [nomeArquivo for nomeArquivo in (os.listdir("./Database/ham")) if len(nomeArquivo) > 20]
arquivosSpam = [nomeArquivo for nomeArquivo in (os.listdir("./Database/spam")) if len(nomeArquivo) > 20]

In [ ]:
secondaryDatabaseOnlySpam = pd.read_csv('onlySpam.csv',delimiter=',').drop(columns=["Unnamed: 0"])

arquivosSpamSecondary = [f"spam{i+len(arquivosSpam)}" for i in range(len(secondaryDatabaseOnlySpam))]

In [ ]:
def carregarEmail(tipo, arquivo):
    caminho = ""

    if tipo == "spam":
        caminho = "./Database/spam"
    if tipo == "ham":
        caminho = "./Database/ham"

    with open(f"{caminho}/{arquivo}", "rb") as file:
        return email.parser.BytesParser(policy=email.policy.default).parse(file)


emailsNormais = [carregarEmail("ham", arquivo) for arquivo in arquivosNormais]
emailsSpam = [carregarEmail("spam", arquivo) for arquivo in arquivosSpam]

In [ ]:
for i in range(len(arquivosNormais)):
    arquivosNormais[i] = f"ham{i}"

for i in range(len(arquivosSpam)):
    arquivosSpam[i] = f"spam{i}"

emailsNome = arquivosNormais + arquivosSpam

emailsNome += arquivosSpamSecondary

In [ ]:
emails = emailsNormais + emailsSpam

aux = emailsNormais[0]

In [ ]:
for i in aux.keys():
    print(f"{i} : {aux[i]}")

print('\n')
print('Conteudo:', aux.get_content())

In [ ]:
def get_email_structure(email):
    if isinstance(email, str):
        return email

    payload = email.get_payload()

    if isinstance(payload, list):
        lista = []
        for sub_email in payload:
            lista.append(get_email_structure(sub_email))

        return "Estruturas do email:", lista

    else:

        return email.get_content_type()


emailsNormaisMultipart = get_email_structure(emailsNormais[634])

In [ ]:
emailsNormaisMultipart

In [ ]:

for email in emailsNormais[634].walk():
    print(email)

In [ ]:
def html_to_text(email):
    try:
        soup = BeautifulSoup(email.get_content(), 'html.parser')
        return soup.text.replace('\n\n', '')
    except:
        return "empty"


def email_to_text(email):
    for emailParts in email.walk():
        if emailParts.get_content_type() not in ['text/plain', 'text/html']:
            continue

        try:
            emailPartsContent = emailParts.get_content()
        except:
            emailPartsContent = str(emailParts.get_payload())

        if emailParts.get_content_type() == 'text/plain':
            return emailPartsContent
        else:
            return html_to_text(emailParts)


'''def email_to_text(email):
    struct = get_email_structure(email)
    for part in email.walk():
        partContentType = part.get_content_type()
        if partContentType not in ['text/plain','text/html']:
            continue
        try:
            partContent = part.get_content()
        except: # in case of encoding issues
            partContent = str(part.get_payload())
        if partContentType == 'text/plain':
            return partContent
        else:
            return html_to_text(part)'''


In [ ]:
email_to_text(emailsNormais[0])

In [ ]:
def removePunctuation(text):
    ponctuation = list(punctuation)

    for i in ponctuation:
        text = text.replace(i, "")

    return text


def wordsPreProcessing(email):
    if email is None:
        return 'empty'

    newText = ""

    stopWords = set(stopwords.words('english')  + list(punctuation) + list(STOPWORDS))
    stopWords.add("subject")
    stem = PorterStemmer()
    wordTokenize = RegexpTokenizer(r'\w+')

    regex = re.compile(r'[\w\.-]+@[\w\.-]+(\.[\w]+)+')

    e = email.split()

    for i in range(0,len(e)):
        if validators.url(e[i]):
            e[i] = "URLLL"

        if re.fullmatch(regex,e[i]):
            e[i] = "EMAILLL"


    for text in word_tokenize(" ".join(e).lower()):
        #print(text)
        text = removePunctuation(text)
        if text not in stopWords and not text.isdigit():

            newText += stem.stem(text) + " "

    return newText

In [ ]:
print(wordsPreProcessing(email_to_text(emails[0])))
print(email_to_text(emails[0]))

In [ ]:
def bagOfWord(emails):
    wordCount = {}

    for email in emails:
        for i in email.split():
            if i not in wordCount.keys():
                wordCount[i] = 1
            else:
                wordCount[i] += 1

    return wordCount

In [ ]:
emailsText = []

for email in emails:
    emailsText.append(wordsPreProcessing
                      (email_to_text(email)))

for email in secondaryDatabaseOnlySpam["text"].array:
    emailsText.append(wordsPreProcessing(email))

print(emailsText[4000])


In [ ]:
wordCount = bagOfWord(emailsText)

wordFrequency = heapq.nlargest(100, wordCount, wordCount.get)

print(wordFrequency)

In [ ]:
tfidfVectorizer = TfidfVectorizer(analyzer="word",max_features=2100)
#tfidfVectorizer = TfidfVectorizer(analyzer="word")

tfidfTransform = tfidfVectorizer.fit_transform(emailsText)

print(tfidfTransform)

In [ ]:
tfidfLabels = tfidfVectorizer.get_feature_names()
tfidfLabels

In [ ]:
dfTfidfArray = pd.DataFrame(data=tfidfTransform.toarray(), index=emailsNome, columns=tfidfLabels)

In [ ]:
dfTfidfArray

In [ ]:
arrayTargetsNormais = [0 for _ in arquivosNormais]

arrayTargetsSpam = [1 for _ in range(len(arquivosSpam)+len(arquivosSpamSecondary))]

targetsArray = arrayTargetsNormais + arrayTargetsSpam

print(len(targetsArray))
print(targetsArray)

In [ ]:
dfTfidfArray.insert(len(dfTfidfArray.columns), "Target", targetsArray, True)
dfTfidfArray = dfTfidfArray.sample(frac=1)
dfTfidfArray


In [ ]:
#dfTfidfArray.to_csv("dataset.csv", sep='\t', encoding='utf-8')

In [ ]:
model = TSNE(n_components=2, random_state=0)
#model = PCA(n_components=50, svd_solver='full')
array_red = model.fit_transform(dfTfidfArray)

df_tsne = pd.DataFrame(array_red)

target = dfTfidfArray["Target"].array

df_tsne['Target'] = target
print(df_tsne)
df_tsne_c1 = df_tsne[df_tsne['Target'] == 0]

df_tsne_c2 = df_tsne[df_tsne['Target'] == 1]

plt.scatter(df_tsne_c1[0].array,df_tsne_c1[1].array,marker='o',color='blue')

plt.scatter(df_tsne_c2[0].array,df_tsne_c2[1].array,marker='o',color='red')

plt.title('Dados')
plt.xlabel('x')
plt.ylabel('y')

plt.show()

In [ ]:
df_features = dfTfidfArray.drop(columns=['Target'])
#df_tsneTarget = df_tsne["Target"].array
#df_tsneFeatures = df_tsne.drop(columns=['Target'])


df_features

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC

def getModel():
      return LogisticRegression(max_iter=200)
#     return DecisionTreeClassifier()
#     return RandomForestClassifier()
#     return LinearSVC()
#     return MultinomialNB()

In [ ]:
X_treino, X_teste, y_treino, y_teste = train_test_split(df_features.values,target,test_size=0.2)
modelo = getModel().fit(X_treino,y_treino)
score = modelo.score(X_teste,y_teste)
#score = modelo.score(["alo"],[1])
score

In [ ]:
scores = cross_val_score(getModel(),df_features.values,target,cv=10)

scores.mean()

In [ ]:
predicoes = cross_val_predict(getModel(), df_features.values, target, cv=10)

cm = confusion_matrix(target,predicoes,labels=[0, 1])

cm_df = pd.DataFrame(cm, columns=[0, 1])

cm_df

In [ ]:
import math

teste = pd.read_csv("spam_ham_dataset.csv").sample(frac=1)
teste2 =pd.read_csv("emails.csv").sample(frac=1)
targetTest2 = teste2["spam"]
teste = teste.drop(columns=["Unnamed: 0","label_num"])
teste = teste.replace({"ham":0,"spam":1})
targetTest = teste["label"]


teste3 = pd.read_csv("enron_spam_data.csv")
teste3 = teste3.drop(columns=["Message ID","Date"])
teste3 = teste3.replace({"ham":0,"spam":1})
teste3 = teste3.drop(columns=["Subject"])
teste3 = teste3.dropna()
teste3 = teste3.sample(frac=1)
targetTest3 = teste3["Spam/Ham"]
#teste
teste2

In [ ]:

emailsTest = []

for email in teste3["Message"]:
    emailsTest.append(wordsPreProcessing(email))


#emailsTest
print("ok")

In [ ]:
emailsTest2 = []

for email in teste2["text"]:
    emailsTest2.append(wordsPreProcessing(email))
    
print("ok")

In [ ]:
emailsTest3 = []

for email in teste["text"]:
    emailsTest3.append(wordsPreProcessing(email))
    
print("ok")

In [ ]:
#APAGAR DEPOIS
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(emailsTest) 
X_train_counts.shape

tf_transformer = TfidfTransformer()
X_train_tfidf = tf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

In [ ]:
##APAGAR DEPOIS
clf = getModel().fit(X_train_tfidf,targetTest3)

X_new_counts = count_vect.transform(emailsTest2) 

X_new_tfidf = tf_transformer.transform(X_new_counts)

predict = clf.predict(X_new_tfidf)
print(len(emailsTest2))
print(predict)
print(targetTest2)
print(teste2)
print(clf.score(X_new_tfidf,targetTest2))

#predicoesTest = cross_val_predict(modelo2, dfTfidfArrayFeaturesTest.values, dfTfidfArrayTargetTest, cv=10)

cmTest = confusion_matrix(targetTest2,predict,labels=[0, 1])

cm_dfTest = pd.DataFrame(cmTest, columns=[0, 1])

cm_dfTest

In [95]:
##APAGAR DEPOIS
clf = getModel().fit(X_train_tfidf,targetTest3)

X_new_counts = count_vect.transform(emailsTest2) 

X_new_tfidf = tf_transformer.transform(X_new_counts)

predict = clf.predict(X_new_tfidf)
print(len(emailsTest2))
print(predict)
print(targetTest2)
print(teste2)
print(clf.score(X_new_tfidf,targetTest2))

#predicoesTest = cross_val_predict(modelo2, dfTfidfArrayFeaturesTest.values, dfTfidfArrayTargetTest, cv=10)

cmTest = confusion_matrix(targetTest2,predict,labels=[0, 1])

cm_dfTest = pd.DataFrame(cmTest, columns=[0, 1])

cm_dfTest

5728
[1 0 1 ... 0 0 0]
1323    1
3371    0
2733    0
5633    0
3477    0
       ..
1826    0
4059    0
4701    0
1186    1
3666    0
Name: spam, Length: 5728, dtype: int64
                                                   text  spam
1323  Subject: pleasure your women - size does matte...     1
3371  Subject: ola oladeji  we are in the process of...     0
2733  Subject: re : enroll in intro to java at produ...     0
5633  Subject: organizational changes  enron is form...     0
3477  Subject: re : stinson vacation plans  stinson ...     0
...                                                 ...   ...
1826  Subject: super saturday iv results !  good aft...     0
4059  Subject: re : gsia visit  duane ,  sorry i wil...     0
4701  Subject: california power 1 / 19 / 00  executi...     0
1186  Subject: professional logo for you now  workin...     1
3666  Subject: norberto  elizabeth ,  i want to disc...     0

[5728 rows x 2 columns]
0.4884776536312849


,0,1
0,2040,2320
1,610,758


In [92]:
#APAGAR DEPOIS
X_new_counts2 = count_vect.transform(emailsTest3) 

X_new_tfidf2 = tf_transformer.transform(X_new_counts2)

predict = clf.predict(X_new_tfidf2)
print(len(emailsTest))
print(predict)
print(targetTest)
print(teste)
print(clf.score(X_new_tfidf2,targetTest))

#predicoesTest = cross_val_predict(modelo2, dfTfidfArrayFeaturesTest.values, dfTfidfArrayTargetTest, cv=10)

cmTest = confusion_matrix(targetTest,predict,labels=[0, 1])

cm_dfTest = pd.DataFrame(cmTest, columns=[0, 1])

cm_dfTest

33345
[1 0 0 ... 0 1 1]
2619    0
4995    1
1938    0
3893    0
3987    0
       ..
1137    0
1354    0
794     0
4346    1
2000    0
Name: label, Length: 5171, dtype: int64
      label                                               text
2619      0  Subject: my vacation\r\ni will be on vacation ...
4995      1  Subject: coca cola , mbna america , nascar par...
1938      0  Subject: tenaska 2 / 01\r\nwe need to forward ...
3893      0  Subject: meter 1603\r\ndaren - meter 1603 for ...
3987      0  Subject: interview schedule for brent trefz\r\...
...     ...                                                ...
1137      0  Subject: hpl nom for sept . 12 , 2000\r\n- - (...
1354      0  Subject: hl & p december\r\njanet . attached i...
794       0  Subject: re : tenaska iv\r\ni ' ll call you on...
4346      1  Subject: how are you ?\r\nnow loading . . .\r\...
2000      0  Subject: mobil chemical - hpl meter # 1256 - e...

[5171 rows x 2 columns]
0.586733707213305


,0,1
0,2559,1113
1,1024,475


In [35]:
dfTfidfArrayTest = pd.DataFrame(data=tfidfTransformTest.toarray(), columns=tfidfVectorizerTest.get_feature_names_out())
dfTfidfArrayTest.insert(len(dfTfidfArrayTest.columns), "Target", targetTest3.array, True)
dfTfidfArrayTest = dfTfidfArrayTest.sample(frac=1)
dfTfidfArrayTest


,aa,abil,abl,absolut,abus,accept,access,accomplish,accord,account,...,year,yesterday,yield,yo,york,young,zero,zimin,zone,Target
1327,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
8194,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.110548,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
14958,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5614,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
703,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29992,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
30790,0.0,0.0,0.00000,0.0,0.0,0.0,0.08161,0.0,0.0,0.0,...,0.062924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
11338,0.0,0.0,0.07552,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
16570,0.0,0.0,0.00000,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [44]:
dfTfidfArrayTest2 = pd.DataFrame(data=tfidfTransformTest2.toarray(), columns=tfidfVectorizerTest2.get_feature_names_out())
dfTfidfArrayTest2.insert(len(dfTfidfArrayTest2.columns), "Target", targetTest.array, True)
dfTfidfArrayTest2 = dfTfidfArrayTest2.sample(frac=1)
dfTfidfArrayTest2


,abdv,abil,abl,absolut,ac,accept,access,accessori,accomplish,accord,...,year,yesterday,york,young,yvett,zero,zivley,zone,zonedubai,Target
3324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1531,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
932,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2029,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2288,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3640,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
5139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:


modelTest = TSNE(n_components=2, random_state=0)
#modelTest = PCA(n_components=50)
array_redTest = modelTest.fit_transform(dfTfidfArrayTest)

df_tsneTest = pd.DataFrame(array_redTest)
df_tsneTest['Target'] = targetTest3.array

df_tsne_c1Test = df_tsneTest[df_tsneTest['Target'] == 0]

df_tsne_c2Test  = df_tsneTest[df_tsneTest['Target'] == 1]

plt.scatter(df_tsne_c1Test[0].array,df_tsne_c1Test[1].array,marker='o',color='blue')

plt.scatter(df_tsne_c2Test[0].array,df_tsne_c2Test[1].array,marker='o',color='red')

plt.title('Dados')
plt.xlabel('x')
plt.ylabel('y')

plt.show()


In [36]:
dfTfidfArrayTarget = dfTfidfArray["Target"].array
dfTfidfArrayFeatures = dfTfidfArray.drop(columns=['Target'])

print(dfTfidfArrayFeatures)

dfTfidfArrayTargetTest = dfTfidfArrayTest["Target"].array
dfTfidfArrayFeaturesTest = dfTfidfArrayTest.drop(columns=['Target'])



print(dfTfidfArrayFeaturesTest)
dfTfidfArrayTargetTest

          abil  abl  absolut      abus  acceler  accept    access  accessori  \
spam82     0.0  0.0      0.0  0.000000      0.0     0.0  0.000000        0.0   
ham1928    0.0  0.0      0.0  0.000000      0.0     0.0  0.000000        0.0   
ham1110    0.0  0.0      0.0  0.000000      0.0     0.0  0.000000        0.0   
ham2350    0.0  0.0      0.0  0.000000      0.0     0.0  0.089848        0.0   
ham888     0.0  0.0      0.0  0.000000      0.0     0.0  0.000000        0.0   
...        ...  ...      ...       ...      ...     ...       ...        ...   
ham1694    0.0  0.0      0.0  0.000000      0.0     0.0  0.000000        0.0   
spam1339   0.0  0.0      0.0  0.000000      0.0     0.0  0.000000        0.0   
spam60     0.0  0.0      0.0  0.000000      0.0     0.0  0.000000        0.0   
spam31     0.0  0.0      0.0  0.123294      0.0     0.0  0.000000        0.0   
spam1978   0.0  0.0      0.0  0.000000      0.0     0.0  0.000000        0.0   

          accord  account  ...  yap    

<PandasArray>
[1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
 ...
 0, 0, 1, 1, 1, 0, 1, 0, 0, 1]
Length: 33345, dtype: int64

In [45]:
dfTfidfArrayTargetTest2 = dfTfidfArrayTest2["Target"].array
dfTfidfArrayFeaturesTest2 = dfTfidfArrayTest2.drop(columns=['Target'])

In [48]:
#from sklearn.svm import LinearSVC

X_treino, X_teste, y_treino, y_teste = train_test_split(dfTfidfArrayFeaturesTest2.values,dfTfidfArrayTargetTest2,test_size=0.2)
X_treinoTest, X_testeTest, y_treinoTest, y_testeTest = train_test_split(dfTfidfArrayFeaturesTest.values,dfTfidfArrayTargetTest,test_size=0.01)
modelo = getModel().fit(X_treino,y_treino)
modelo2 = getModel().fit(X_treinoTest,y_treinoTest)
predict = modelo.predict(X_teste)
#score = modelo.score(X_treinoTest,y_treinoTest)
#score2 = modelo2.score(X_treino,y_treino)

cmTest = confusion_matrix(y_teste,predict)
cm_dfTest = pd.DataFrame(cmTest, columns=[0, 1])

cm_dfTest

,0,1
0,733,9
1,10,283


In [48]:
#predicoesTest = cross_val_predict(modelo2, dfTfidfArrayFeaturesTest.values, dfTfidfArrayTargetTest, cv=10)

#cmTest = confusion_matrix(dfTfidfArrayTargetTest,predicoesTest,labels=[0, 1])

#cm_dfTest = pd.DataFrame(cmTest, columns=[0, 1])

#cm_dfTest